# 7.2 ODE I

#### Today's class:

* Non-linear equations
    - Relaxation method
    - Binary search - bisection method
    - Newton-Raphson
* Skydiver problem: Falling body with drag
    - Equation of motion
    - Solve ODE explicit
    - Solve with library
    

## Non-linear equations
### Relaxation method

Find solution to transcendental equation 
$$
x = 2-e^{-x}
$$
by relaxation method. Guess initial value for $x$, enter into RHS, calculate new LHS and calculate RHS again, and hope that this converges.

Start with a plot. Then experiment with a couple of initial values. 

In [ ]:
%pylab ipympl

In [ ]:
ifig=1;close(ifig);figure(ifig)
x = linspace(-4,4,100)
plot(x,2-e**(-x))
plot(x,x,'--')

In [ ]:
x0 = 4
x = 1.1*x0
while abs(x-x0) > 1e-2:
    x0 = x
    print(x0)
    x = 2-e**(-x0)
print(x)

We can quickly see some problems. This is not a very general method. 

The method can fail in even more dramatic ways:

$$
x = e^{1-x^2}
$$

In [ ]:
ifig=2;close(ifig);figure(ifig)
x = linspace(-4,4,100)
rhs = lambda x : e**(1-x**2)
plot(x,rhs(x))
plot(x,x,'--')

In [ ]:
x0 = 2
x = 1.1*x0
for i in range(10):
    x0 = x
    print(x0)
    x = rhs(x)
print(x)

No matter where you start you can't find the root as you are oscillating back and forth. Interestingly, rearranging will lead to success:
$$
x = \sqrt{(1-\log{x})}
$$

In [ ]:
rhs = lambda x : sqrt(1 - log(x))

In [ ]:
x0 = 2.
x = 1.1*x0
while abs(x-x0) > 1e-3:
    x0 = x
    #print(x0)
    x = rhs(x)
print(x)

### Binary search - bisection method

In [ ]:
rhs = lambda x : e**(1-x**2)
xrange = linspace(0,2,10)

In [ ]:
xx = rhs(xrange)-xrange
ind = where(xx[:-1]*xx[1:]<0)[0][0]
xrange = linspace(xrange[4],xrange[5],10)
xrange[ind]

### Newton-Raphson

Using the definition of the derivative 
$$
f'(x) = \frac{f(x)}{\Delta x}
$$
we can improve on an initial estimate of $x$ by
$$
x' = x - \Delta x =  x - \frac{f(x)}{f'(x) }
$$

In [ ]:
x = linspace(-6,6,10000)
rhs = lambda x : e**(1-x**2)
G = lambda x : rhs(x)-x

In [ ]:
# from class notes 4.2 
def deriv2(f,x,h):
    dfdx = (f(x+h) - f(x-h)) / (2*h)
    return dfdx

In [ ]:
# check how G looks like, what are we trying to find the root of?
ifig=2;close(ifig);figure(ifig)
plot(x,G(x),':')
axhline(color='k',linestyle='dashed',lw=0.75)
xlabel('$x_\mathrm{n+1}$')
ylabel('$G$')

In [ ]:
xx=1.5; h=0.01   # we start with an estimate for x_0 = 0.5
plot(xx,G(xx),'h')

In [ ]:
# one NR iteration, repeat until satisfactory solution is found
# x is the estimate for x_n+1 and accepted as solution if 
# G(xx) < eps1 and delta < eps2, where eps1 and eps2 are 
# suitably small limits

x = xx
delta = G(x)/deriv2(G,x,h)
xx = x - delta
print(xx,G(xx),delta)
plot(xx,G(xx),'h')

In [ ]:
from scipy import optimize

In [ ]:
optimize.newton(G, 90)

In [ ]:
FF = lambda x: 2-e**(-x) - x

In [ ]:
ifig=6;close(ifig);figure(ifig)
x = linspace(-2,4,100)
plot(x,FF(x))
hlines(0,-2,4,lw=0.75)

In [ ]:
optimize.newton(FF, [-1.19,1.73])

## Skydiver problem: Falling body with drag

A falling body - say a skydiver - will increase speed when she jumps off the plane because she is accelerated by the earth's gravity. However, the speed will not increase forever. In addition to the gravitational force the sky diver will feel the drag force due to air resistance.

What is the terminal velocity of the sky diver? This will depend on the balance of two forces: the gravitational force and the drag force that describes the air resistance


### Equation of motion
The equation of motion for the velocity is $v = a t +v_0$ where $a$ is the acceration and $v_0$ the initial velocity. But this is of course just a special case of the more general case
$$\frac{d\vec{p}}{dt} = \sum \vec{F}_i .$$
where $\vec{p}$ is the momentum and $\vec{F}_i$ is one of several forces that may act, like gravitational force and friction.

#### Forces
We will consider a 1D motion in the vertical direction, and therefore consider the scalar equations.

#### Gravity

$$F = - m g$$
where $m$ is the mass of the body and $g$ is the magnitude of the gravitational acceleration.

#### Air drag

The   expression for the drag force is 
$$F_D = \frac{1}{2} C_D \rho v^2 A ,$$ where $C_D \approx 0.3$ is the drag coefficient, $v$ velocity of object experiencing the drag, $\rho = 1.224 \mathrm{kg/m^3}$ the density of air the object is experiencing the drag in. $A$ is the cross-sectional area of the object, for a skydiver assume $A = 1 m^2$, and their weight is $80\mathrm{kg}$.


#### Equation of motion for sky diver


$$ m \frac{dv}{dt} = -mg + \frac{1}{2} C_D \rho v^2 A $$
or, with $$k = \frac{1}{2} \frac{C_\mathrm{D} \rho A}{m}$$ we just have
$$ \frac{dv}{dt} = -g +  k v^2.$$

#### In class activity:
* what is the unit of k
* calculate k using the units package

In [ ]:
import astropy.units as u

In [ ]:
Cd  = 0.3
rho = 1.225 *u.kg/u.m**3
A   = 1 * u.m**2
m = 80 * u.kg
k = 0.5*Cd*rho*A/m
k

In order to solve this differential equation on a compute we use the Euler step method. The most simple solution scheme would be the following: 

$$\frac{v_\mathrm{n+1} - v_\mathrm{n}}{h} = -g + kv_\mathrm{n}^2$$

where the subscript $n$ indicates subsequents steps in time, and $h$ is the choosen time step length $\Delta t$. Solving for $v_\mathrm{n+1}$ yields:

$$ v_\mathrm{n+1} = v_\mathrm{n} + h(kv_\mathrm{n}^2 -g) $$

Finally, we just need some appropriate initial conditions, such as $v_\mathrm{0} = 0$.

What we want to get is the function $v(t)$. How will it likely look like? Initially the velocity will increase as the graviational acceleration dominates. Ultimately the quadratic drag term in $v$ will become noticable. In fact, there is an equillibrium solution then the drag force equals the gravitational force. Equillibrium means that nothing changes, i.e. $$\frac{dv}{dt}=0$$ 

In that case from the differential equation above we find that the terminal velocity is  $$v_\mathrm{T} = \sqrt{\frac{g}{k}}$$
Calculating this terminal velocity will provide us with a scale for the problem.

In [ ]:
g = 9.8 * u.m/u.s**2
v_t = sqrt(g/k)
print(v_t.to('km/h'))

Shed the units going forward:

In [ ]:
vt = v_t.value  # m/s
g = g.value
k = k.value 

#### Solve ODE
We must integrate both the velocity as well as the distance according to 
$$
\frac{dh}{dt} = v
$$


In [ ]:
h0   = 2000   # jump-off height [m]
h_p  = 300    # end of free flight, pull parachute height [m]

In [ ]:
def rhs_sdiver(x,dt):
    'Evaluate RHS for skydiver problem, advance one time step'
    h = x[0] + dt*x[1]
    v = x[1] + dt*(k*x[1]**2 - g)
    return [h,v]

We must also estimate a reasonable time step. 

In [ ]:
dh = 10  # typical spatial step at terminal velocity [m]
dt = dh / vt
print('%5.3f'%dt,'s')

In [ ]:
hv_all = []
hv = array([h0,0])
while hv[0] > h_p:
    hv = rhs_sdiver(hv,dt)
    hv_all.append(hv)

In [ ]:
hv_10=array(hv_all)

In [ ]:
hv_plot_10=hv_10.T
close(1),figure(1)
plot(hv_plot_10[0],hv_plot_10[1],label='dh= 10m')
plot((hv_plot_10[0][0],hv_plot_10[0][-1]),(-vt,-vt),'k-',lw=0.75)
xlabel('h / [m]')
ylabel('v / [km/s]')
legend(loc=1)
xlim(2100,250)

#### Convergence test

In [ ]:
t_max = 15

dh = [10,100, 300]
close(2);figure(2)
for this_dh in dh:
    dt = this_dh / vt
    tv_all = [[0,0]] # each element is [t, v]
    hv = array([0,0]); t = 0
    while t < t_max:
        hv = rhs_sdiver(hv,dt)
        t += dt
        tv_all.append([t,hv[1]])
    tv = array(tv_all).T
    plot(tv[0],tv[1],'-o',label='dh='+str(this_dh)+'m')
xlabel('t / [s]')
ylabel('v / [m/s]')
legend(loc=1)


In [ ]:
close(3);figure(3)
dh_range = [5,10,20,40,80,160]
#dh_range = [2,100]
for dh in dh_range[::-1]:
    dt = dh / vt
    print('time step %5.3f'%dt,'s')
    hv_all = []
    hv = array([2000,0])
    while hv[0] > h_p:
        hv = rhs_sdiver(hv,dt)
        hv_all.append(hv)
    hv_plot = array(hv_all).T    
    plot(hv_plot[0],log10(hv_plot[1]+vt),label='dh='+str(dh)+'m')
xlabel('h [m]')
ylabel('$ \log (v-v_\mathrm{t}) \mathrm{\ [km/s]}$')
legend(loc=1)
xlim(2100,250)

### Solution for system with odeint
Plot H and v as a function of t.

In [ ]:
def rhs_sdiver_odeint(y,t,k,g):
    'Evaluate RHS for skydiver problem, advance one time step'
    hrhs = y[1]
    vrhs = k*y[1]**2 - g
    return [hrhs,vrhs]


In [ ]:
from scipy import integrate

In [ ]:
t=linspace(0,25,25)
y = integrate.odeint(rhs_sdiver_odeint,[h0,0],t, args=(k,g))

In [ ]:
close(5);figure(5)
plot(t,y.T[0])
ylabel('height / m')
xlabel('time / s')

In [ ]:
close(6);figure(6)
plot(t,y.T[1])
ylabel('velocity / m/s')
xlabel('time / s')

### Solving with a library - try another solver
The documentation says _For new code, use_ `scipy.integrate.solve_ivp` _to solve a differential equation._ OK, ... let's try that ...

In [ ]:
# integrate.solve_ivp?

In [ ]:
rhs_ff_sivp = lambda x,y: 2*x   # note that for this solver the calling 
                                # sequence is different than for odeint!
sol = integrate.solve_ivp(rhs_ff_sivp,[0,7],[0],t_eval=linspace(0,7,8)) # note the different call 
                                                 # arguments compared to odeing

This libary provides a range of solvers, including the _LSODA_ solver used in `odeint`. Try the option `method='LSODA'`.

In [ ]:
sol

In [ ]:
close(7);figure(7)
plot(sol.t,sol.y[0],'h-.',label='scipy.integrate.solve_ivp')
plot(sol.t,sol.t**2)
legend()